# Hybrid Architecture Experiments (MODEL-6)

This notebook explores hybrid architectures that combine different model paradigms for RUL prediction.

## Experiments

1. **TCN -> Transformer Hybrid**: Use TCN for local feature extraction followed by Transformer for global context
2. **2D CNN + State Space Model (SSM)**: Use CNN for spectrogram features combined with SSM for temporal modeling

## Architecture Comparisons

| Hybrid | Feature Extractor | Temporal Model | Input Type |
|--------|------------------|----------------|------------|
| TCN-Transformer | Multi-scale TCN | Transformer Encoder | Raw 1D signals |
| CNN-SSM | 2D CNN Backbone | State Space Model | Spectrograms |

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from dataclasses import dataclass, field
from typing import Literal, Optional, List

# Add project root to path
PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f'TensorFlow version: {tf.__version__}')
print(f'NumPy version: {np.__version__}')

In [ ]:
# Import existing model components
from src.models.pattern1 import (
    create_tcn_transformer_lstm,
    create_tcn_transformer_transformer,
    StemConfig,
    TCNConfig,
    print_model_summary as print_model_summary_p1,
)
from src.models.pattern1.stem import DualChannelStem
from src.models.pattern1.tcn import DualChannelTCN, TCNEncoder
from src.models.pattern1.attention import BidirectionalCrossAttention, AttentionConfig
from src.models.pattern1.aggregator import TransformerAggregator, TransformerAggregatorConfig
from src.models.pattern1.model import RULHead, TemporalDownsampler

from src.models.pattern2 import (
    create_pattern2_lstm,
    create_pattern2_transformer,
    CNN2DBackboneConfig,
    print_model_summary as print_model_summary_p2,
)
from src.models.pattern2.backbone import DualChannelCNN2DBackbone
from src.models.pattern2.model import LateFusion

from src.training.config import TrainingConfig, compile_model, build_callbacks
from src.training.metrics import rmse, mae, phm08_score

from src.data.loader import XJTUBearingLoader
from src.data.rul_labels import generate_rul_labels
from src.features.stft import extract_spectrogram

print('Imports successful!')

---

## Hybrid 1: TCN -> Transformer

This hybrid combines the best of both worlds:
- **TCN**: Excellent at capturing local patterns with multi-scale receptive fields via dilated convolutions
- **Transformer**: Better at capturing global dependencies through self-attention

### Architecture
```
Input (batch, 32768, 2)
    |
    v
DualChannelStem (Conv1D feature extraction)
    |
    v
Multi-Resolution TCN (local patterns, d=1,2,4,8,16,32)
    |
    v
Temporal Downsampling (32768 -> 512 for memory efficiency)
    |
    v
Transformer Encoder (global context, 4 layers)
    |
    v
RUL Head
```

### Key Differences from Pattern 1
- More Transformer layers (4 vs 2)
- Deeper TCN stack
- No cross-attention (simpler pipeline)
- Higher downsampling factor for longer Transformer context

In [ ]:
@dataclass
class TCNTransformerHybridConfig:
    """Configuration for TCN -> Transformer hybrid model."""
    input_length: int = 32768
    num_channels: int = 2
    
    # Stem config
    stem_filters: int = 64
    
    # TCN config (local feature extraction)
    tcn_filters: int = 64
    tcn_dilations: list = field(default_factory=lambda: [1, 2, 4, 8, 16, 32])
    tcn_kernel_size: int = 3
    tcn_dropout: float = 0.1
    
    # Downsampling
    downsample_factor: int = 64  # Higher factor for longer Transformer context
    
    # Transformer config (global context)
    transformer_layers: int = 4  # More layers than Pattern 1
    transformer_heads: int = 4
    transformer_key_dim: int = 64
    transformer_ff_dim: int = 128
    transformer_dropout: float = 0.1
    
    # RUL head
    hidden_dim: int = 64
    dropout_rate: float = 0.1

In [ ]:
def build_tcn_transformer_hybrid(
    config: Optional[TCNTransformerHybridConfig] = None,
    name: str = 'tcn_transformer_hybrid'
) -> keras.Model:
    """Build TCN -> Transformer hybrid model.
    
    This architecture uses TCN for local feature extraction
    followed by Transformer for global temporal modeling.
    """
    if config is None:
        config = TCNTransformerHybridConfig()
    
    # Input
    inputs = keras.Input(
        shape=(config.input_length, config.num_channels),
        name='input'
    )
    
    # Per-channel stem
    stem = DualChannelStem(
        config=StemConfig(filters=config.stem_filters),
        share_weights=False,
        name='stem'
    )
    h_stem, v_stem = stem(inputs)
    
    # TCN encoding (local patterns)
    tcn = DualChannelTCN(
        config=TCNConfig(
            filters=config.tcn_filters,
            kernel_size=config.tcn_kernel_size,
            dilations=config.tcn_dilations,
            dropout_rate=config.tcn_dropout,
        ),
        share_weights=False,
        name='tcn'
    )
    h_tcn, v_tcn = tcn((h_stem, v_stem))
    
    # Concatenate channels before downsampling
    concat = layers.Concatenate(axis=-1, name='channel_concat')([h_tcn, v_tcn])
    
    # Temporal downsampling (for memory-efficient Transformer)
    downsampler = TemporalDownsampler(
        factor=config.downsample_factor,
        mode='avg',
        name='downsample'
    )
    downsampled = downsampler(concat)
    
    # Transformer encoder (global context)
    transformer = TransformerAggregator(
        config=TransformerAggregatorConfig(
            num_layers=config.transformer_layers,
            num_heads=config.transformer_heads,
            key_dim=config.transformer_key_dim,
            ff_dim=config.transformer_ff_dim,
            dropout_rate=config.transformer_dropout,
            use_cls_token=True,
            pooling='cls',
        ),
        name='transformer'
    )
    aggregated = transformer(downsampled)
    
    # RUL head
    rul_head = RULHead(
        hidden_dim=config.hidden_dim,
        dropout_rate=config.dropout_rate,
        name='rul_head'
    )
    output = rul_head(aggregated)
    
    model = keras.Model(inputs=inputs, outputs=output, name=name)
    return model

In [ ]:
# Build TCN-Transformer hybrid
config_tcn_transformer = TCNTransformerHybridConfig(
    stem_filters=64,
    tcn_filters=64,
    tcn_dilations=[1, 2, 4, 8, 16, 32],
    downsample_factor=64,
    transformer_layers=4,
    transformer_heads=4,
)

model_tcn_transformer = build_tcn_transformer_hybrid(config_tcn_transformer)

print('=== TCN -> Transformer Hybrid ===')
print_model_summary_p1(model_tcn_transformer)

In [ ]:
# Full model summary
model_tcn_transformer.summary()

---

## Hybrid 2: 2D CNN + State Space Model (SSM)

State Space Models (like S4, Mamba) are an alternative to Transformers for sequence modeling:
- **Linear time complexity** O(n) vs O(n²) for Transformers
- **Better at very long sequences** due to continuous-time formulation
- **Recurrent inference** makes them efficient at test time

### Architecture
```
Input Spectrograms (batch, 128, 128, 2)
    |
    v
DualChannel CNN Backbone (spatial features)
    |
    v
Late Fusion (concat H, V embeddings)
    |
    v
State Space Layer (temporal dynamics)
    |
    v
RUL Head
```

### Simplified SSM Implementation

We implement a simplified S4-inspired layer using:
- Learnable state transition matrix A
- Discretization via bilinear transform
- Parallel scan for efficient computation

In [ ]:
class SimpleSSMLayer(keras.layers.Layer):
    """Simplified State Space Model layer.
    
    Implements a basic linear state space model:
        h_t = A * h_{t-1} + B * x_t
        y_t = C * h_t + D * x_t
    
    Uses diagonal state matrix for efficiency.
    """
    
    def __init__(
        self,
        state_dim: int = 64,
        output_dim: int = 64,
        dropout_rate: float = 0.1,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.state_dim = state_dim
        self.output_dim = output_dim
        self.dropout_rate = dropout_rate
        
        self.input_proj = None
        self.state_proj = None
        self.output_proj = None
        self.dropout = None
        self.layer_norm = None
        
    def build(self, input_shape):
        input_dim = input_shape[-1]
        
        # B: Input to state projection
        self.input_proj = layers.Dense(
            self.state_dim,
            use_bias=False,
            name=f'{self.name}_B'
        )
        
        # A: State transition (diagonal, initialized near identity for stability)
        # Use sigmoid to keep values in (0, 1) for stability
        self.A_logit = self.add_weight(
            name='A_logit',
            shape=(self.state_dim,),
            initializer=keras.initializers.Constant(2.0),  # sigmoid(2) ~ 0.88
            trainable=True
        )
        
        # C: State to output projection
        self.state_proj = layers.Dense(
            self.output_dim,
            use_bias=False,
            name=f'{self.name}_C'
        )
        
        # D: Skip connection
        self.output_proj = layers.Dense(
            self.output_dim,
            name=f'{self.name}_D'
        )
        
        self.dropout = layers.Dropout(self.dropout_rate)
        self.layer_norm = layers.LayerNormalization(name=f'{self.name}_ln')
        
        super().build(input_shape)
        
    def call(self, inputs, training=None):
        """Process sequence through SSM.
        
        Args:
            inputs: (batch, seq_len, features)
            
        Returns:
            (batch, output_dim) - aggregated output
        """
        # A in (0, 1) for stability
        A = keras.ops.sigmoid(self.A_logit)
        
        # Project input
        Bx = self.input_proj(inputs)  # (batch, seq, state_dim)
        
        # Recurrent scan (simplified parallel implementation)
        # For efficiency, we use a simple loop here
        # In practice, you'd use parallel scan for GPU efficiency
        batch_size = keras.ops.shape(inputs)[0]
        seq_len = keras.ops.shape(inputs)[1]
        
        # Initialize state
        h = keras.ops.zeros((batch_size, self.state_dim))
        outputs = []
        
        # Simple unrolled scan for short sequences
        # Note: For production, use tf.scan or parallel associative scan
        for t in range(keras.ops.convert_to_numpy(seq_len)):
            h = A * h + Bx[:, t, :]
            outputs.append(h)
        
        # Stack outputs
        all_states = keras.ops.stack(outputs, axis=1)  # (batch, seq, state_dim)
        
        # Project to output and add skip connection
        y = self.state_proj(all_states) + self.output_proj(inputs)
        
        # Layer norm and dropout
        y = self.layer_norm(y)
        y = self.dropout(y, training=training)
        
        # Return last state as summary (or could use mean pooling)
        return y[:, -1, :]
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'state_dim': self.state_dim,
            'output_dim': self.output_dim,
            'dropout_rate': self.dropout_rate,
        })
        return config

In [ ]:
# Alternative: Bidirectional SSM with GRU-like gating
class GatedSSMLayer(keras.layers.Layer):
    """Gated State Space Model layer with bidirectional processing.
    
    Combines SSM dynamics with GRU-style gating for better gradient flow.
    """
    
    def __init__(
        self,
        units: int = 64,
        bidirectional: bool = True,
        dropout_rate: float = 0.1,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.units = units
        self.bidirectional = bidirectional
        self.dropout_rate = dropout_rate
        
    def build(self, input_shape):
        input_dim = input_shape[-1]
        
        # Forward GRU-like layer (approximates SSM with gating)
        self.gru_fwd = layers.GRU(
            self.units,
            return_sequences=True,
            name=f'{self.name}_gru_fwd'
        )
        
        if self.bidirectional:
            self.gru_bwd = layers.GRU(
                self.units,
                return_sequences=True,
                go_backwards=True,
                name=f'{self.name}_gru_bwd'
            )
        
        # Linear mixing (SSM-inspired)
        out_dim = self.units * 2 if self.bidirectional else self.units
        self.mix = layers.Dense(self.units, name=f'{self.name}_mix')
        
        self.dropout = layers.Dropout(self.dropout_rate)
        self.layer_norm = layers.LayerNormalization(name=f'{self.name}_ln')
        
        super().build(input_shape)
        
    def call(self, inputs, training=None):
        # Forward pass
        fwd = self.gru_fwd(inputs)
        
        if self.bidirectional:
            # Backward pass
            bwd = self.gru_bwd(inputs)
            # Reverse to align with forward
            bwd = keras.ops.flip(bwd, axis=1)
            # Concatenate
            combined = keras.ops.concatenate([fwd, bwd], axis=-1)
        else:
            combined = fwd
        
        # Linear mixing
        mixed = self.mix(combined)
        
        # Norm and dropout
        mixed = self.layer_norm(mixed)
        mixed = self.dropout(mixed, training=training)
        
        # Return last timestep
        return mixed[:, -1, :]
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'bidirectional': self.bidirectional,
            'dropout_rate': self.dropout_rate,
        })
        return config

In [ ]:
@dataclass
class CNNSSMHybridConfig:
    """Configuration for 2D CNN + SSM hybrid model."""
    # Input (spectrograms)
    spectrogram_height: int = 128
    spectrogram_width: int = 128
    
    # CNN backbone
    cnn_filters: list = field(default_factory=lambda: [32, 64, 128, 256])
    cnn_kernel_sizes: list = field(default_factory=lambda: [3, 3, 3, 3])
    share_weights: bool = True
    
    # Fusion
    fusion_mode: str = 'concat'
    
    # SSM config
    ssm_type: str = 'gated'  # 'simple' or 'gated'
    ssm_units: int = 64
    ssm_bidirectional: bool = True
    ssm_dropout: float = 0.1
    
    # RUL head
    hidden_dim: int = 64
    dropout_rate: float = 0.1

In [ ]:
def build_cnn_ssm_hybrid(
    config: Optional[CNNSSMHybridConfig] = None,
    name: str = 'cnn_ssm_hybrid'
) -> keras.Model:
    """Build 2D CNN + State Space Model hybrid.
    
    Uses CNN for spatial feature extraction from spectrograms,
    followed by SSM for temporal modeling.
    """
    if config is None:
        config = CNNSSMHybridConfig()
    
    # Input
    inputs = keras.Input(
        shape=(config.spectrogram_height, config.spectrogram_width, 2),
        name='spectrogram_input'
    )
    
    # 2D CNN backbone
    backbone = DualChannelCNN2DBackbone(
        config=CNN2DBackboneConfig(
            filters=config.cnn_filters,
            kernel_sizes=config.cnn_kernel_sizes,
        ),
        share_weights=config.share_weights,
        name='backbone'
    )
    h_emb, v_emb = backbone(inputs)
    
    # Late fusion
    fusion = LateFusion(
        fusion_mode=config.fusion_mode,
        name='fusion'
    )
    fused = fusion((h_emb, v_emb))
    
    # Expand to sequence for SSM (single spectrogram = length 1)
    # In practice, you might have multiple spectrograms per bearing
    fused_seq = layers.Reshape((1, -1), name='expand_seq')(fused)
    
    # State Space Model
    if config.ssm_type == 'simple':
        ssm = SimpleSSMLayer(
            state_dim=config.ssm_units,
            output_dim=config.ssm_units,
            dropout_rate=config.ssm_dropout,
            name='ssm'
        )
    else:  # gated
        ssm = GatedSSMLayer(
            units=config.ssm_units,
            bidirectional=config.ssm_bidirectional,
            dropout_rate=config.ssm_dropout,
            name='ssm'
        )
    aggregated = ssm(fused_seq)
    
    # RUL head
    x = layers.Dense(config.hidden_dim, activation='gelu', name='rul_hidden')(aggregated)
    x = layers.Dropout(config.dropout_rate, name='rul_dropout')(x)
    output = layers.Dense(1, activation='relu', name='rul_output')(x)
    
    model = keras.Model(inputs=inputs, outputs=output, name=name)
    return model

In [ ]:
# Build CNN-SSM hybrid
config_cnn_ssm = CNNSSMHybridConfig(
    cnn_filters=[32, 64, 128, 256],
    ssm_type='gated',
    ssm_units=64,
    ssm_bidirectional=True,
)

model_cnn_ssm = build_cnn_ssm_hybrid(config_cnn_ssm)

print('=== 2D CNN + SSM Hybrid ===')
print_model_summary_p2(model_cnn_ssm)

In [ ]:
# Full model summary
model_cnn_ssm.summary()

---

## Load Data for Experiments

In [ ]:
# Load bearing data
loader = XJTUBearingLoader()

# Load a single bearing for experiments
condition = '35Hz12kN'
bearing_id = 'Bearing1_1'

print(f'Loading {bearing_id} from {condition}...')
signals, file_paths = loader.load_bearing(condition, bearing_id)

# Generate RUL labels
rul_labels = generate_rul_labels(
    num_files=len(file_paths),
    strategy='piecewise_linear',
    max_rul=125
)

print(f'Signal shape: {signals.shape}')  # (num_files, 32768, 2)
print(f'RUL labels shape: {rul_labels.shape}')

In [ ]:
# Generate spectrograms for CNN-SSM model
print('Generating spectrograms...')

n_samples = min(50, len(signals))  # Use subset for speed
spectrograms = np.array([
    extract_spectrogram(signals[i])
    for i in range(n_samples)
])

print(f'Spectrogram shape: {spectrograms.shape}')  # (n, 128, 128, 2)

In [ ]:
# Prepare data for both models
X_raw = signals[:n_samples]
X_spec = spectrograms
y = rul_labels[:n_samples].reshape(-1, 1)

# Train/val split
split_idx = int(0.8 * n_samples)

X_raw_train, X_raw_val = X_raw[:split_idx], X_raw[split_idx:]
X_spec_train, X_spec_val = X_spec[:split_idx], X_spec[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

print(f'Training samples: {len(y_train)}')
print(f'Validation samples: {len(y_val)}')

---

## Training Experiments

In [ ]:
# Training configuration
training_config = TrainingConfig(
    learning_rate=1e-3,
    batch_size=4,
    epochs=10,  # Quick demo
)

def train_and_evaluate(model, X_train, X_val, y_train, y_val, name):
    """Train model and return metrics."""
    # Compile
    compile_model(model, training_config)
    
    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=training_config.epochs,
        batch_size=training_config.batch_size,
        verbose=1,
    )
    
    # Evaluate
    y_pred = model.predict(np.vstack([X_train, X_val]), verbose=0).flatten()
    y_true = np.vstack([y_train, y_val]).flatten()
    
    metrics = {
        'model': name,
        'rmse': rmse(y_true, y_pred),
        'mae': mae(y_true, y_pred),
        'phm08': phm08_score(y_true, y_pred),
        'train_loss': history.history['loss'][-1],
        'val_loss': history.history['val_loss'][-1],
    }
    
    return history, metrics

In [ ]:
# Train TCN-Transformer Hybrid
print('=' * 50)
print('Training TCN-Transformer Hybrid...')
print('=' * 50)

# Use smaller model for demo
config_small = TCNTransformerHybridConfig(
    stem_filters=32,
    tcn_filters=32,
    tcn_dilations=[1, 2, 4, 8],
    downsample_factor=128,  # Higher downsampling
    transformer_layers=2,
    transformer_heads=2,
    transformer_key_dim=32,
    transformer_ff_dim=64,
)
model_tcn_transformer_small = build_tcn_transformer_hybrid(config_small, 'tcn_transformer_small')

history_tcn, metrics_tcn = train_and_evaluate(
    model_tcn_transformer_small,
    X_raw_train, X_raw_val,
    y_train, y_val,
    'TCN-Transformer'
)

In [ ]:
# Train CNN-SSM Hybrid
print('=' * 50)
print('Training CNN-SSM Hybrid...')
print('=' * 50)

# Use smaller model for demo
config_small_ssm = CNNSSMHybridConfig(
    cnn_filters=[16, 32, 64, 128],
    ssm_type='gated',
    ssm_units=32,
)
model_cnn_ssm_small = build_cnn_ssm_hybrid(config_small_ssm, 'cnn_ssm_small')

history_ssm, metrics_ssm = train_and_evaluate(
    model_cnn_ssm_small,
    X_spec_train, X_spec_val,
    y_train, y_val,
    'CNN-SSM'
)

---

## Compare Architectures on Validation Set

In [ ]:
# Comparison table
results = pd.DataFrame([metrics_tcn, metrics_ssm])
print('\n=== Architecture Comparison ===')
print(results.to_string(index=False))

In [ ]:
# Training curves comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training loss
ax1 = axes[0]
ax1.plot(history_tcn.history['loss'], 'b-', label='TCN-Transformer Train', linewidth=2)
ax1.plot(history_tcn.history['val_loss'], 'b--', label='TCN-Transformer Val', linewidth=2)
ax1.plot(history_ssm.history['loss'], 'r-', label='CNN-SSM Train', linewidth=2)
ax1.plot(history_ssm.history['val_loss'], 'r--', label='CNN-SSM Val', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Huber Loss')
ax1.set_title('Training Curves')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Metrics bar chart
ax2 = axes[1]
x = np.arange(3)
width = 0.35
metrics_names = ['RMSE', 'MAE', 'Val Loss']
tcn_values = [metrics_tcn['rmse'], metrics_tcn['mae'], metrics_tcn['val_loss']]
ssm_values = [metrics_ssm['rmse'], metrics_ssm['mae'], metrics_ssm['val_loss']]

ax2.bar(x - width/2, tcn_values, width, label='TCN-Transformer', color='steelblue')
ax2.bar(x + width/2, ssm_values, width, label='CNN-SSM', color='coral')
ax2.set_ylabel('Value')
ax2.set_title('Metrics Comparison')
ax2.set_xticks(x)
ax2.set_xticklabels(metrics_names)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Prediction comparison
y_pred_tcn = model_tcn_transformer_small.predict(X_raw, verbose=0).flatten()
y_pred_ssm = model_cnn_ssm_small.predict(X_spec, verbose=0).flatten()
y_true = y.flatten()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time series
ax1 = axes[0]
ax1.plot(y_true, 'k-', label='Ground Truth', linewidth=2)
ax1.plot(y_pred_tcn, 'b--', label='TCN-Transformer', linewidth=2, alpha=0.7)
ax1.plot(y_pred_ssm, 'r--', label='CNN-SSM', linewidth=2, alpha=0.7)
ax1.set_xlabel('Sample Index')
ax1.set_ylabel('RUL')
ax1.set_title('RUL Predictions Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Scatter
ax2 = axes[1]
ax2.scatter(y_true, y_pred_tcn, alpha=0.6, label='TCN-Transformer', color='steelblue')
ax2.scatter(y_true, y_pred_ssm, alpha=0.6, label='CNN-SSM', color='coral')
max_val = max(y_true.max(), y_pred_tcn.max(), y_pred_ssm.max())
ax2.plot([0, max_val], [0, max_val], 'k--', label='Perfect')
ax2.set_xlabel('True RUL')
ax2.set_ylabel('Predicted RUL')
ax2.set_title('Prediction Scatter Plot')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

---

## Document Findings and Architecture Trade-offs

In [ ]:
# Model statistics
print('=== Model Statistics ===')
print('\nTCN-Transformer Hybrid:')
print_model_summary_p1(model_tcn_transformer_small)

print('\nCNN-SSM Hybrid:')
print_model_summary_p2(model_cnn_ssm_small)

## Conclusions and Recommendations

### TCN-Transformer Hybrid

**Pros:**
- Works directly on raw signals (no preprocessing)
- TCN provides efficient multi-scale local feature extraction
- Transformer captures global dependencies
- Dilated convolutions give large receptive field with fewer parameters

**Cons:**
- Requires significant downsampling before Transformer (memory constraints)
- Longer training time due to sequence length
- May lose fine-grained temporal information after downsampling

**Best for:**
- When raw signal access is important
- When multi-scale patterns matter
- When computational resources are available

---

### CNN-SSM Hybrid

**Pros:**
- CNN is highly efficient for spectrogram processing
- SSM has linear time complexity O(n)
- More memory efficient than Transformer-based models
- Spectrograms provide time-frequency representation

**Cons:**
- Requires spectrogram preprocessing
- Information loss during spectrogram generation
- Simplified SSM may not capture all temporal dynamics

**Best for:**
- When memory is limited
- When spectrograms are pre-computed
- When linear scaling with sequence length is needed

---

### Recommendations

1. **For production deployment**: CNN-SSM hybrid is more efficient and scalable
2. **For maximum accuracy**: TCN-Transformer hybrid may capture more nuanced patterns
3. **For ensemble**: Combine both models for potentially better predictions
4. **Future work**: 
   - Implement proper S4/Mamba SSM for better temporal modeling
   - Explore attention mechanisms in SSM (like Mamba's selective state spaces)
   - Test with multi-spectrogram sequences for longer context

---

## Summary Table

| Aspect | TCN-Transformer Hybrid | CNN-SSM Hybrid |
|--------|----------------------|----------------|
| Input | Raw signals (32768, 2) | Spectrograms (128, 128, 2) |
| Feature Extractor | TCN (dilated convolutions) | 2D CNN (spatial) |
| Temporal Model | Transformer Encoder | State Space Model |
| Time Complexity | O(n²) in attention | O(n) linear |
| Memory | High (needs downsampling) | Lower |
| Preprocessing | None | STFT required |
| Interpretability | Attention weights | State dynamics |